In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("learn.csv")

In [3]:
del data["Unnamed: 0"]

In [4]:
data.insert_time = pd.to_datetime(data.insert_time)
data.company_enter_month = pd.to_datetime(data.company_enter_month)

In [36]:
x = data.copy()

In [37]:
c1 = pd.get_dummies(x.income_type)

In [38]:
x = pd.concat([x, c1], axis = 1)

In [39]:
del x['income_type']

In [40]:
x = x[["application_id", "loan_limit", "loan_rate", "credit_score", "company_enter_month","insert_time",
                  "desired_amount", "EARNEDINCOME", "is_applied"]]

In [10]:
y = x.groupby("application_id").max()

In [11]:
y = y.loc[y.is_applied == 1]

In [12]:
appyes = y.index 

In [41]:
x = x.loc[x.application_id.isin(appyes)]

In [42]:
x = x.sort_values(by = "application_id")

In [43]:
x["predict"] = 0

In [44]:
x = x.sort_values(by = ["application_id", "loan_rate"], ascending = [True, True])

In [45]:
x["more"] = False

In [46]:
x["sum"] = 0

In [47]:
x.iloc[0, 11] = x.iloc[0, 1]

In [48]:
x

,application_id,loan_limit,loan_rate,credit_score,company_enter_month,insert_time,desired_amount,EARNEDINCOME,is_applied,predict,more,sum
4955966,14,12000000.0,6.6,590.0,2021-06-01,2022-04-15 01:02:52,20000000.0,1,1.0,0,False,12000000
4955883,14,12000000.0,6.6,590.0,2021-06-01,2022-04-15 01:02:52,20000000.0,1,1.0,0,False,0
4955890,14,12000000.0,7.1,590.0,2021-06-01,2022-04-15 01:02:52,20000000.0,1,0.0,0,False,0
4955887,14,12000000.0,7.9,590.0,2021-06-01,2022-04-15 01:02:52,20000000.0,1,0.0,0,False,0
4955970,14,12000000.0,8.0,590.0,2021-06-01,2022-04-15 01:02:52,20000000.0,1,0.0,0,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2591389,2167826,22000000.0,16.9,560.0,2017-05-01,2022-03-25 11:57:48,5000000.0,1,0.0,0,False,0
2591428,2167826,22000000.0,16.9,560.0,2017-05-01,2022-03-25 11:57:48,5000000.0,1,0.0,0,False,0
2591393,2167826,13000000.0,17.4,560.0,2017-05-01,2022-03-25 11:57:48,5000000.0,1,1.0,0,False,0
2591390,2167826,5000000.0,18.7,560.0,2017-05-01,2022-03-25 11:57:48,5000000.0,1,0.0,0,False,0


In [49]:
for i in range (1, len(x)):
    x.iloc[i, 11] = x.iloc[i, 1]
    if (x.iloc[i, 0] == x.iloc[i-1, 0]):
        x.iloc[i, 11] += x.iloc[i-1, 11]

In [53]:
x["more"] = x["sum"] > x.desired_amount

In [64]:
x.iloc[0, 9] = 1

In [69]:
for i in range(1, len(x)):
    if (x.iloc[i, 0] == x.iloc[i-1, 0]):
        if (not x.iloc[i, 10]):
            x.iloc[i, 9] = 1
        elif (not x.iloc[i-1, 10]):
            x.iloc[i, 9] = 1
    else:
        x.iloc[i, 9] = 1

In [71]:
x.to_csv("rational_pred.csv")

In [72]:
from sklearn.metrics import f1_score

In [73]:
x

,application_id,loan_limit,loan_rate,credit_score,company_enter_month,insert_time,desired_amount,EARNEDINCOME,is_applied,predict,more,sum
4955966,14,12000000.0,6.6,590.0,2021-06-01,2022-04-15 01:02:52,20000000.0,1,1.0,1,False,12000000
4955883,14,12000000.0,6.6,590.0,2021-06-01,2022-04-15 01:02:52,20000000.0,1,1.0,1,True,24000000
4955890,14,12000000.0,7.1,590.0,2021-06-01,2022-04-15 01:02:52,20000000.0,1,0.0,0,True,36000000
4955887,14,12000000.0,7.9,590.0,2021-06-01,2022-04-15 01:02:52,20000000.0,1,0.0,0,True,48000000
4955970,14,12000000.0,8.0,590.0,2021-06-01,2022-04-15 01:02:52,20000000.0,1,0.0,0,True,60000000
...,...,...,...,...,...,...,...,...,...,...,...,...
2591389,2167826,22000000.0,16.9,560.0,2017-05-01,2022-03-25 11:57:48,5000000.0,1,0.0,1,True,22000000
2591428,2167826,22000000.0,16.9,560.0,2017-05-01,2022-03-25 11:57:48,5000000.0,1,0.0,0,True,44000000
2591393,2167826,13000000.0,17.4,560.0,2017-05-01,2022-03-25 11:57:48,5000000.0,1,1.0,0,True,57000000
2591390,2167826,5000000.0,18.7,560.0,2017-05-01,2022-03-25 11:57:48,5000000.0,1,0.0,0,True,62000000


In [107]:
x["weird"] = (x.is_applied - x.predict == 1)

In [110]:
x["weird"].sum()

230642

In [202]:
y = x.loc[x.weird]

In [204]:
weirdid = y.index

In [206]:
d = data.loc[weirdid]

In [208]:
d.to_csv("이상한놈들.csv")

In [151]:
model2 = CatBoostClassifier(iterations=2,
                           depth=5,
                           learning_rate=1,
                           loss_function='Logloss',
                           verbose=True)

In [153]:
del x["application_id"]
del x["more"]
del x["sum"]

In [189]:
xxtr = x[x['insert_time']<'2022-05-16']
xxtt = x[x['insert_time']>= '2022-05-16']

In [190]:
del xxtr["insert_time"]
del xxtt["insert_time"]

In [192]:
xxtrl = xxtr["weird"] * 1

In [193]:
xxttl = xxtt["weird"] * 1

In [194]:
del xxtr["weird"]
del xxtt["weird"]

In [195]:
del xxtr["is_applied"]
del xxtt["is_applied"]

In [167]:
kk = x.weird.mean()

In [196]:
xxtrain = Pool(
    data = xxtr,
    label = xxtrl,
    weight = xxtrl * (1-2*kk) + kk
)

In [197]:
model2.fit(xxtrain)

0:	learn: 0.5843210	total: 178ms	remaining: 178ms
1:	learn: 0.5512695	total: 307ms	remaining: 0us


In [198]:
ppp = model2.predict(xxtt)

In [200]:
f1_score(xxttl, ppp)

0.17483582023694835

In [126]:
y = x.loc[x.weird]

In [127]:
y

,application_id,loan_limit,loan_rate,credit_score,company_enter_month,insert_time,desired_amount,EARNEDINCOME,is_applied,predict,more,sum,weird
4955884,14,10000000.0,12.9,590.0,2021-06-01,2022-04-15 01:02:52,20000000.0,1,1.0,0,True,82000000,True
5814803,36,27000000.0,12.3,640.0,2018-12-01,2022-04-22 14:12:10,30000000.0,1,1.0,0,True,88000000,True
5814765,36,10000000.0,12.5,640.0,2018-12-01,2022-04-22 14:12:10,30000000.0,1,1.0,0,True,98000000,True
3330938,38,10000000.0,7.1,540.0,2021-07-01,2022-04-01 14:18:25,5000000.0,0,1.0,0,True,40000000,True
3588714,48,3000000.0,15.0,570.0,2021-09-01,2022-04-04 12:56:43,10000000.0,1,1.0,0,True,45000000,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5339419,2167787,31000000.0,7.7,682.0,2021-04-01,2022-04-18 23:24:45,6.0,0,1.0,0,True,60000000,True
8046457,2167803,10000000.0,19.9,630.0,2022-01-01,2022-05-12 14:26:29,3.0,1,1.0,0,True,82000000,True
4459325,2167811,12000000.0,8.0,620.0,2022-01-01,2022-04-11 14:49:23,1000000.0,1,1.0,0,True,60000000,True
3405174,2167817,1000000.0,19.9,580.0,2018-11-01,2022-04-02 10:10:31,2000000.0,0,1.0,0,True,36000000,True


In [144]:
trainy = Pool(
    data = Y_train,
    label = trainy_label,
    weight = trainy_label * (1-2*k) + k
)

In [74]:
f1_score(x.is_applied, x.predict)

0.5123673207708854

In [75]:
from sklearn.metrics import confusion_matrix

In [77]:
confusion_matrix(x.is_applied, x.predict).ravel()

array([3822162,  386207,  230642,  324069])

In [78]:
from catboost import CatBoostClassifier, Pool

In [133]:
model = CatBoostClassifier(iterations=2,
                           depth=5,
                           learning_rate=1,
                           loss_function='Logloss',
                           verbose=True)

In [134]:
X_train = x[x['insert_time']<'2022-05-16']
X_test = x[x['insert_time']>= '2022-05-16']

In [135]:
del X_train ["insert_time"]
del X_test ["insert_time"]

In [136]:
del X_train ["sum"]
del X_test ["sum"]

In [137]:
del X_train["more"]
del X_test ["more"]

In [138]:
train_label = X_train["is_applied"]
test_label = X_test["is_applied"]

In [139]:
del X_train["is_applied"]
del X_test["is_applied"]

In [140]:
del X_train["application_id"]
del X_test["application_id"]

In [141]:
del X_train["weird"]
del X_test["weird"]

In [142]:
k = x.is_applied.mean()

In [143]:
train = Pool(
    data = X_train,
    label = train_label,
    weight = train_label * (1-2*k) + k
)

In [145]:
model.fit(train)

0:	learn: 0.5030316	total: 201ms	remaining: 201ms
1:	learn: 0.4820259	total: 310ms	remaining: 0us


In [146]:
pp = model.predict(X_test)

In [147]:
f1_score(test_label, pp)

0.48169246313388336

In [148]:
model.get_feature_importance()

array([ 7.36268323,  4.05583113, 10.33484238,  2.02747862,  7.49613976,
        3.41214522, 65.31087966])

In [124]:
confusion_matrix(test_label, pp).ravel()

array([730734,  58141,    699, 106119])